# <center> ERP TRANSACTIONAL ANALYSIS AND VISUALIZATION WITH PYTHON

## OVERVIEW

The purpose of this document is to provide an example of data analysis and visualization using Python coding techniques. Manipulating data with code instead of relational spreadsheets such as Excel allows increased computational power, flexibility and superior graphing capability.

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import calendar
import csv
import random

%matplotlib inline
sns.set()

pd.set_option('display.float_format', lambda x: '%.2f' % x)


from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
init_notebook_mode(connected=True)
import plotly.graph_objects as go

from mlxtend.frequent_patterns import apriori, association_rules

import requests
import io

In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
df = pd.read_csv('transactions.csv')

In [4]:
df['inv_date'] = pd.to_datetime(df['inv_date'])

In [5]:
df['year'] = df['inv_date'].dt.year
df['month'] = df['inv_date'].dt.month
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])

new = df['disc']
new2 = []

for n in new:
    if n <= 10:
        new2.append('3. Retail')
    elif (n > 10) and (n < 30):
        new2.append("2. Mid")
    else:
        new2.append('1. Premier')
df['pricing_structure'] = new2

df = df[df['year'] != 2020] # remove incomplete year

## DATA

The practice data used in this example was obtained from an open source data [repository](https://www.kaggle.com/) popular with data science students. It consists of 103498 rows of 13 variables representing nearly 10 years worth of parts sales invoices. The data is presented in 'molten' format - each variable has its own column, every observation has its own row, subjects are divided into their component observations and each value has its own cell. <br>
This dataset does not provide a comprehensive illustration of a parts supply chain. For a more complete picture we would also need purchase orders and GRV dates, supplier lead time guidance, sales order dates for backorder analysis, DC and ship-to locations to measure logistical efficiency, and goods return data to measure quality and sales accuracy. <br>
 

In [6]:
df

,inv_date,invoice_number,account,part_number,qty,sell_price,total_sale_value,disc,cost_price,total_cost_value,profit,year,month,pricing_structure
0,2010-04-01,INV7811696,ELEMERA,P522817,1.00,1232.14,1232.14,36,1059.64,1059.64,172.50,2010,Apr,1. Premier
1,2010-07-01,INV7811682,ELEMERA,P522817,1.00,788.57,788.57,36,678.17,678.17,110.40,2010,Jul,1. Premier
2,2010-02-25,INV7811426,ELEMERA,P522817,1.00,788.57,788.57,36,678.17,678.17,110.40,2010,Feb,1. Premier
3,2010-02-25,INV7811426,ELEMERA,P857011,1.00,63.06,63.06,36,54.24,54.24,8.83,2010,Feb,1. Premier
4,2010-03-29,INV7811269,ELEMERA,P522817,1.00,788.57,788.57,36,678.17,678.17,110.40,2010,Mar,1. Premier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103493,2010-11-24,INV7809808,DAVES CONSTRUCTION EQUIPMENT,P931108,1.00,107.11,107.11,0,53.56,53.56,53.56,2010,Nov,3. Retail
103494,2010-11-24,INV7809808,DAVES CONSTRUCTION EQUIPMENT,P345492,2.00,75.94,151.88,0,37.97,75.94,75.94,2010,Nov,3. Retail
103495,2015-12-17,INV7796840,THE RAILROADERS,P599748,1.00,724.13,724.13,0,362.07,362.07,362.07,2015,Dec,3. Retail
103496,2016-07-03,INV7795983,BOBS SERVICING & DELIVERY,P599748,1.00,957.17,957.17,0,478.59,478.59,478.59,2016,Jul,3. Retail


## CUSTOMER GROUPING BY PRICE STRUCTURE


We can factor engineer new values from the original dataset. Creating a visual representation for over 100 different customers is difficult. The stacked bar chart below shows sales activity by customer discount structure. Hover over each bar for more detailed information.

In [7]:
saleval = pd.pivot_table(df, index = ['year','pricing_structure'], values = ['total_sale_value','profit'], aggfunc = {'total_sale_value':'sum','profit':'sum'})
saleval = saleval.reset_index(drop = False)

In [8]:
fig = px.bar(saleval, x = 'year', y = 'total_sale_value', color = 'pricing_structure', title='Parts Sales by Customer Discount Structure')
fig.update_xaxes(type='category') # fix x axis to make the years look nice
fig.show()

Sales growth has been underpinned by consistently increasing sales to premier customers. Midrange sales have also increased, possibly indicating a change in pricing strategy.

## CUSTOMER ACTIVITY ANALYSIS

Scatter plots are designed to visualize the relationship between two values. It is important to confirm that a customer sales activity justifies their discount structure. The graph below measures each customers sales activity by aggregate line items sold against total profit generated. <br>
Scatter plots can also be used for inventory profitability analysis.

In [9]:
customers = pd.pivot_table(df, index = ['account'], values = ['invoice_number','part_number','total_sale_value','profit','pricing_structure'], aggfunc = {'invoice_number':'count','part_number':'count','total_sale_value':'sum','profit':'sum','pricing_structure':'first'})
customers = customers.rename(columns = {'invoice_number':'total_invoices', 'part_number':'transaction_lines'})
customers = customers[['total_invoices','transaction_lines','total_sale_value','profit','pricing_structure']]
customers = customers.reset_index(drop=False)
customers = customers.sort_values(by=['pricing_structure'], ascending = True)

In [10]:
fig = px.scatter(customers, x='transaction_lines', y='profit', log_x=True, log_y = True, size_max=100, hover_name = 'account', color='pricing_structure', title='Total Revenue vs Transaction Lines per Customer')
fig.show()

Retail customers exclusively inhabit the lower volume space, but several customers on higher discounts appear to be spending less than expected. Further research is required to see if some accounts can justify their negotiated pricing.

## TARGETED CUSTOMER INSIGHTS

The stacked bar chart below a breakdown of sales results for each premier customer by month. Use the range finder to zoom in on any period of interest.

In [11]:
saleval_premier = df.drop(df[df.pricing_structure != '1. Premier'].index) # only investigate premier customers
saleval_premier['mth'] = saleval_premier['inv_date'].dt.month # add numerical month value

saleval_premier = pd.pivot_table(saleval_premier, index = ['year','mth', 'account'], values = ['total_sale_value','profit'], aggfunc = {'total_sale_value':'sum','profit':'sum'})
saleval_premier = saleval_premier.reset_index(drop = False)

saleval_premier['year'] = saleval_premier['year'].astype(str)
saleval_premier['mth'] = saleval_premier['mth'].astype(str)
saleval_premier['month_year'] = saleval_premier['year'] + '_' + saleval_premier['mth']

In [12]:
fig = px.bar(saleval_premier, x='month_year', y='total_sale_value', color = 'account', title='Premier Account Monthly Sales')


fig.update_xaxes(rangeslider_visible=True)
fig.show()

A more in depth assessment of sales figures reveals consistently two key accounts with consistently high volumes. Four accounts have a favourable pricing structure that is not supported by recent sales activity.

## CONCLUSION & FURTHER RECOMMENDATIONS FOR FURTHER ANALYSIS

This document provides a brief introduction to the potential of analysis and presentations with Python programming. I look forward to utilizing these skills in a real life environment.